# AI plays Snake Game
   <h4 style='color:purple;'>Remember that this is only a starting point!</h4><br><br>
In order to make this agent better you may want to change the following (<span style='color:blue;'>mybe not!</span>)
<ol>
    <li>Reward Rate</li>
    <li>Batch Size</li>
    <li>Learning Rate</li>
    <li>Memory Size</li>
    <li>A better strategy to choose the the data we want to remember! (hint: mybe those with greater reward point)</li>
    <br>
    <p style='color:red;'>Note: You may want to create your data and then feed it to the agent!</p>
<ol>

In [1]:
import pygame
import numpy as np
import random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Snake:
    
    WINDOW_WIDTH = 400
    WINDOW_HEIGHT = 400
    snake_img = pygame.image.load("slice_of_snake.png")
    food_img = pygame.image.load("food.png")
    
    def __init__(self):
        self.DEF_LOCATION = np.full((20,20),0) 
        self.DEF_LOCATION[10][2]=6
        self.DEF_LOCATION[10][3]=6
        self.DEF_LOCATION[10][4]=6
        self.head = [10,4]
        self.tail = [10,2]
        self.food = [random.randint(0,9),random.randint(0,19)]
        self.DEF_LOCATION[self.food[0],self.food[1]]=20
        self.location = self.DEF_LOCATION
        self.alive = True
        self.food_found = 0
    
    
    def update(self):
        if self.location[self.head[0]][self.head[1]] == 6:
            if self.location[self.head[0]][(self.head[1]+1)%20] == 0:
                self.location[self.head[0]][(self.head[1]+1)%20] = 6
                self.head[1]=(self.head[1]+1)%20
            elif self.location[self.head[0]][(self.head[1]+1)%20] == 20:
                food = self.new_food()
                self.location[self.head[0]][(self.head[1]+1)%20] = 6
                self.location[self.head[0]][(self.head[1]+2)%20] = 6
                self.head[1]=(self.head[1]+2)%20
                self.food_found+=20
            else:
                self.alive = False
        elif self.location[self.head[0]][self.head[1]] == 8:
            if self.location[(self.head[0]-1)%20][self.head[1]] == 0:
                self.location[(self.head[0]-1)%20][self.head[1]] = 8
                self.head[0]=(self.head[0]-1)%20
            elif self.location[(self.head[0]-1)%20][self.head[1]] == 20:
                food = self.new_food()
                self.location[(self.head[0]-1)%20][self.head[1]] = 8
                self.location[(self.head[0]-2)%20][self.head[1]] = 8
                self.head[0]=(self.head[0]-2)%20
                self.food_found+=20
            else:
                self.alive = False
        elif self.location[self.head[0]][self.head[1]] == 2:
            if self.location[(self.head[0]+1)%20][self.head[1]] == 0:
                self.location[(self.head[0]+1)%20][self.head[1]] = 2
                self.head[0]=(self.head[0]+1)%20
            elif self.location[(self.head[0]+1)%20][self.head[1]] == 20:
                food = self.new_food()
                self.location[(self.head[0]+1)%20][self.head[1]] = 2
                self.location[(self.head[0]+2)%20][self.head[1]] = 2
                self.head[0]=(self.head[0]+2)%20
                self.food_found+=20
            else:
                self.alive = False
        elif self.location[self.head[0]][self.head[1]] == 4:
            if self.location[self.head[0]][(self.head[1]-1)%20] == 0:
                self.location[self.head[0]][(self.head[1]-1)%20] = 4
                self.head[1]=(self.head[1]-1)%20
            elif self.location[self.head[0]][(self.head[1]-1)%20] == 20:
                food = self.new_food()
                self.location[self.head[0]][(self.head[1]-1)%20] = 4
                self.location[self.head[0]][(self.head[1]-2)%20] = 4
                self.head[1]=(self.head[1]-2)%20
                self.food_found+=20
            else:
                self.alive = False
        
        
        if self.location[self.tail[0]][self.tail[1]]==6:
            self.location[self.tail[0]][self.tail[1]]=0
            self.tail[1]=(self.tail[1]+1)%20
        elif self.location[self.tail[0]][self.tail[1]]==8:
            self.location[self.tail[0]][self.tail[1]]=0
            self.tail[0]=(self.tail[0]-1)%20
        elif self.location[self.tail[0]][self.tail[1]]==4:
            self.location[self.tail[0]][self.tail[1]]=0
            self.tail[1]=(self.tail[1]-1)%20
        elif self.location[self.tail[0]][self.tail[1]]==2:
            self.location[self.tail[0]][self.tail[1]]=0
            self.tail[0]=(self.tail[0]+1)%20
    
    
    def new_food(self):
        x = random.randint(0,19)
        y = random.randint(0,19)
        while self.location[x][y] != 0:
            x = random.randint(0,19)
            y = random.randint(0,19)
        self.food = [x,y]
        self.location[x][y]=20
            
        
    def move(self, direction):
        self.location[self.head[0]][self.head[1]]=direction

        
    def show(self):
        for i in range(20):
            for j in range(20):
                if self.location[i][j] == 20:
                    gameDisplay.blit(self.food_img,(j*20,i*20))
                elif self.location[i][j] != 0:
                    gameDisplay.blit(self.snake_img,(j*20,i*20))

In [3]:
import os
from collections import deque
import random
import numpy as np
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from math import sqrt

Using TensorFlow backend.


In [4]:
state_size = 400  # amp of the game! 
action_size = 4 # up right down left

batch_size = 250

n_episodes = 5000 #number of games that our agent has to play

output_dir = "model_output/snake-game"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [5]:
class DQNAgent():
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen=3000)
        
        self.gamma = 0.95
        
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.learning_rate = 0.001
        
        self.model = self.build_model()
        
    def build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) 
        return np.argmax(act_values[0]) 

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size) 
        for state, action, reward, next_state, done in minibatch: 
            target = reward 
            if not done: 
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0])) 
            target_f = self.model.predict(state) 
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) 
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
agent = DQNAgent(state_size, action_size)
done = False
for e in range(n_episodes):
    
    
    # state restart!
    pygame.init()
    newGame = Snake()
    gameDisplay = pygame.display.set_mode((newGame.WINDOW_WIDTH, newGame.WINDOW_HEIGHT))
    pygame.display.set_caption("Snake")
    clock = pygame.time.Clock() 
    #end of state restart
    state = np.reshape(newGame.location, [1, state_size])
    #game!
    time = 0
    while True:
        action = agent.act(state)

        if action == 0:
            newGame.move("8")
        elif action == 1:
            newGame.move("2")
        elif action == 2:
            newGame.move("4")
        elif action == 3:
            newGame.move("6")

        gameDisplay.fill((255,255,255))
        newGame.update()
        newGame.show()
        pygame.display.update()

        next_state= np.reshape(newGame.location, [1, state_size])
        ps=newGame.head
        pf=newGame.food
        reward = (40-(sqrt(((pf[1]-ps[1])**2)+((pf[0]-ps[0])**2))) if newGame.alive else -10) + newGame.food_found
        done = newGame.alive


        agent.remember(state, action, reward, next_state, done)

        state = next_state

        time+=1
        clock.tick(5)
        if not newGame.alive:
            #print("Game Over")
            del newGame
            break
    print("episode: {}/{}, time-score= {}, e: {:.2}".format(e,n_episodes, time , agent.epsilon))
    time = 0
    if len(agent.memory) > batch_size:
            agent.replay(batch_size)
            print("############Agent Training!")
    if e % 100 == 0:
        agent.save(output_dir+ "weights_"+ '{:04d}'.format(e)+".hdf5")
    pygame.quit()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
episode: 0/5000, time-score= 1, e: 1.0
episode: 1/5000, time-score= 1, e: 1.0
episode: 2/5000, time-score= 4, e: 1.0
episode: 3/5000, time-score= 5, e: 1.0
episode: 4/5000, time-score= 2, e: 1.0
episode: 5/5000, time-score= 5, e: 1.0
episode: 6/5000, time-score= 11, e: 1.0
episode: 7/5000, time-score= 5, e: 1.0
episode: 8/5000, time-score= 1, e: 1.0
episode: 9/5000, time-score= 1, e: 1.0
episode: 10/5000, time-score= 1, e: 1.0
episode: 11/5000, time-score= 1, e: 1.0
episode: 12/5000, time-score= 3, e: 1.0
episode: 13/5000, time-score= 2, e: 1.0
episode: 14/5000, time-score= 5, e: 1.0
episode: 15/5000, time-score= 1, e: 1.0
episode: 16/5000, time-score= 2, e: 1.0
episode: 17/5000, time-score= 3, e: 1.0
episode: 18/5000, time-score= 4, e: 1.0
episode: 19/5000, time-score= 4, e: 1.0
episode: 20/5000, time-score= 1, e: 1.0
episode: 21/5000, time-score= 1, e: 1.0
episode: 22/5000, time-score= 4, e: 1.0
episode:

episode: 137/5000, time-score= 3, e: 0.69
############Agent Training!
episode: 138/5000, time-score= 4, e: 0.69
############Agent Training!
episode: 139/5000, time-score= 4, e: 0.69
############Agent Training!
episode: 140/5000, time-score= 9, e: 0.68
############Agent Training!
episode: 141/5000, time-score= 6, e: 0.68
############Agent Training!
episode: 142/5000, time-score= 3, e: 0.68
############Agent Training!
episode: 143/5000, time-score= 3, e: 0.67
############Agent Training!
episode: 144/5000, time-score= 1, e: 0.67
############Agent Training!
episode: 145/5000, time-score= 10, e: 0.67
############Agent Training!
episode: 146/5000, time-score= 2, e: 0.66
############Agent Training!
episode: 147/5000, time-score= 24, e: 0.66
############Agent Training!
episode: 148/5000, time-score= 11, e: 0.66
############Agent Training!
episode: 149/5000, time-score= 22, e: 0.65
############Agent Training!
episode: 150/5000, time-score= 5, e: 0.65
############Agent Training!
episode: 151/500

episode: 254/5000, time-score= 6, e: 0.39
############Agent Training!
episode: 255/5000, time-score= 2, e: 0.38
############Agent Training!
episode: 256/5000, time-score= 1, e: 0.38
############Agent Training!
episode: 257/5000, time-score= 12, e: 0.38
############Agent Training!
episode: 258/5000, time-score= 18, e: 0.38
############Agent Training!
episode: 259/5000, time-score= 1, e: 0.38
############Agent Training!
episode: 260/5000, time-score= 4, e: 0.37
############Agent Training!
episode: 261/5000, time-score= 4, e: 0.37
############Agent Training!
episode: 262/5000, time-score= 5, e: 0.37
############Agent Training!
episode: 263/5000, time-score= 5, e: 0.37
############Agent Training!
episode: 264/5000, time-score= 17, e: 0.37
############Agent Training!
episode: 265/5000, time-score= 10, e: 0.37
############Agent Training!
episode: 266/5000, time-score= 8, e: 0.36
############Agent Training!
episode: 267/5000, time-score= 8, e: 0.36
############Agent Training!
episode: 268/500

############Agent Training!
episode: 371/5000, time-score= 8, e: 0.21
############Agent Training!
episode: 372/5000, time-score= 9, e: 0.21
############Agent Training!
episode: 373/5000, time-score= 33, e: 0.21
############Agent Training!
episode: 374/5000, time-score= 17, e: 0.21
############Agent Training!
episode: 375/5000, time-score= 17, e: 0.21
############Agent Training!
episode: 376/5000, time-score= 13, e: 0.21
############Agent Training!
episode: 377/5000, time-score= 11, e: 0.21
############Agent Training!
episode: 378/5000, time-score= 12, e: 0.21
############Agent Training!
episode: 379/5000, time-score= 1, e: 0.21
############Agent Training!
episode: 380/5000, time-score= 9, e: 0.21
############Agent Training!
episode: 381/5000, time-score= 9, e: 0.2
############Agent Training!
episode: 382/5000, time-score= 3, e: 0.2
############Agent Training!
episode: 383/5000, time-score= 29, e: 0.2
############Agent Training!
episode: 384/5000, time-score= 10, e: 0.2
############Agen

episode: 487/5000, time-score= 49, e: 0.12
############Agent Training!
episode: 488/5000, time-score= 34, e: 0.12
############Agent Training!
episode: 489/5000, time-score= 6, e: 0.12
############Agent Training!
episode: 490/5000, time-score= 8, e: 0.12
############Agent Training!
episode: 491/5000, time-score= 8, e: 0.12
############Agent Training!
episode: 492/5000, time-score= 3, e: 0.12
############Agent Training!
episode: 493/5000, time-score= 34, e: 0.12
############Agent Training!
episode: 494/5000, time-score= 21, e: 0.12
############Agent Training!
episode: 495/5000, time-score= 29, e: 0.12
############Agent Training!
episode: 496/5000, time-score= 23, e: 0.11
############Agent Training!
episode: 497/5000, time-score= 14, e: 0.11
############Agent Training!
episode: 498/5000, time-score= 13, e: 0.11
############Agent Training!
episode: 499/5000, time-score= 22, e: 0.11
############Agent Training!
episode: 500/5000, time-score= 32, e: 0.11
############Agent Training!
episode: 5

episode: 602/5000, time-score= 124, e: 0.067
############Agent Training!
episode: 603/5000, time-score= 45, e: 0.067
############Agent Training!
episode: 604/5000, time-score= 69, e: 0.067
############Agent Training!
episode: 605/5000, time-score= 1, e: 0.066
############Agent Training!
episode: 606/5000, time-score= 7, e: 0.066
############Agent Training!
episode: 607/5000, time-score= 13, e: 0.066
############Agent Training!
episode: 608/5000, time-score= 17, e: 0.065
############Agent Training!
episode: 609/5000, time-score= 16, e: 0.065
############Agent Training!
episode: 610/5000, time-score= 5, e: 0.065
############Agent Training!
episode: 611/5000, time-score= 10, e: 0.064
############Agent Training!
episode: 612/5000, time-score= 138, e: 0.064
############Agent Training!
episode: 613/5000, time-score= 7, e: 0.064
############Agent Training!
episode: 614/5000, time-score= 55, e: 0.063
############Agent Training!
episode: 615/5000, time-score= 19, e: 0.063
############Agent Trai

############Agent Training!
episode: 717/5000, time-score= 5, e: 0.038
############Agent Training!
episode: 718/5000, time-score= 165, e: 0.038
############Agent Training!
episode: 719/5000, time-score= 9, e: 0.038
############Agent Training!
episode: 720/5000, time-score= 20, e: 0.037
############Agent Training!
episode: 721/5000, time-score= 15, e: 0.037
############Agent Training!
episode: 722/5000, time-score= 16, e: 0.037
############Agent Training!
episode: 723/5000, time-score= 33, e: 0.037
############Agent Training!
episode: 724/5000, time-score= 10, e: 0.037
############Agent Training!
episode: 725/5000, time-score= 1, e: 0.036
############Agent Training!
episode: 726/5000, time-score= 8, e: 0.036
############Agent Training!
episode: 727/5000, time-score= 8, e: 0.036
############Agent Training!
episode: 728/5000, time-score= 72, e: 0.036
############Agent Training!
episode: 729/5000, time-score= 3, e: 0.036
############Agent Training!
episode: 730/5000, time-score= 66, e: 0.0